In [ ]:
# 문장 분리기
!pip install kss

In [ ]:
# 띄어쓰기 검사기
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

# 오류 발생
# 구글링 결과 
# !pip uninstall imgaug
# !pip install imgaug==0.2.5
# 위 코드로 해결된다고 해서 시도했으나 안 됨

In [ ]:
# 맞춤법 검사기
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
# 정규화
!pip install soynlp

In [ ]:
# 외래어 사전
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" -o confused_loanwords.txt

In [ ]:
# 형태소 분리기
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [ ]:
import pandas as pd

df = pd.read_csv('./기아+기아.csv', encoding='utf-8', index_col=0)
df

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()
df.isnull().sum()

In [ ]:
import kss

# 문장단위 분리
sentence_tokenized_text = []
for comment in df.comments:
    comment = comment.strip()
    for sent in kss.split_sentences(comment):
        sentence_tokenized_text.append(sent.strip())

In [ ]:
# 특수문자 설정
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [ ]:
# 특수문자 대체재 설정
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", 
"√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", 
'“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 
'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 
'π': 'pi', }

In [ ]:
# 특수문자 정리
def clean_punc(text, punct, mapping):
    
		# 특수문자를 특수문자 대체제로 변환
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    # 특수문자 분리
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    return text.strip()

In [ ]:
# 함수 실행
cleaned_corpus = []
for sent in sentence_tokenized_text:
    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

In [ ]:
# 결과 확인
for i in range(0, 10):
    print(cleaned_corpus[i])

In [ ]:
import re

# 정규표현식 문자열 처리
def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) # 특수문자 제거
        review = re.sub(r'\d+','', str(texts[i])) # 숫자제거
        review = review.lower() # 소문자 변환
        review = re.sub(r'<[^>]+>','',review) # html 태그 제거
        review = re.sub(r'\s+', ' ', review) # 공백 제거
        review = re.sub(r"^\s+", '', review) # 문자열 앞의 공백 제거
        review = re.sub(r'\s+$', '', review) # 문자열 뒤의 공백 제거
        corpus.append(review)
    return corpus

In [ ]:
# 결과 확인
basic_preprocessed_corpus = clean_text(cleaned_corpus)

for i in range(0, 10):
    print(basic_preprocessed_corpus[i])

In [ ]:
from pykospacing import spacing
from hanspell import spell_checker
from soynlp.normalizer import *

# 외래어 사전 불러오기
lownword_map = {}
lownword_data = open('/content/confused_loanwords.txt', 'r', encoding='utf-8')
lines = lownword_data.readlines()

# 외래어 사전 재구성
for line in lines:
    line = line.strip()
    miss_spell = line.split('\t')[0] # 잘못된 외래어 분리
    ori_word = line.split('\t')[1] # 수정 후 외래어 분리
		# 맞춤법이 틀린 외래어를 key로 설정, 수정 후 외래어를 value로 설정해 딕셔너리 생성
    lownword_map[miss_spell] = ori_word

In [ ]:
def spell_check_text(texts):
    corpus = []
    for sent in texts:
        spaced_text = spacing(sent) # 띄어쓰기
        spelled_sent = spell_checker.check(sent) # 맞춤법 검사
        checked_sent = spelled_sent.checked 
				# 정규화 ex) 와하하하하하하핫 = 와하하핫
        normalized_sent = repeat_normalize(checked_sent) 
        for lownword in lownword_map:
						# 외래어 사전 딕셔너리 key에 해당하는 단어가 있을 경우 그에 맞는 value 단어로 변환
            normalized_sent = normalized_sent.replace(lownword, lownword_map[lownword])
        corpus.append(normalized_sent)
    return corpus

In [ ]:
# 함수 실행
spell_preprocessed_corpus = spell_check_text(basic_preprocessed_corpus)

In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

# 품사 설정
# 일반 명사, 고유 명사, 의존 명사, 동사, 형용사, 보조 용언, 일반 부사, 접속 부사, 동사 파생 접미사, 형용사 파생 접미사
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent): # 띄어쓰기 기준 분리, 형태소 + 품사 형태 반환
            for morph in word.morphs:
                if morph.tag in significant_tags:
										# 형태소 + / + 품사 + ' '
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus

In [ ]:
# 함수 적용
pos_tagged_corpus = pos_text(spell_preprocessed_corpus)

for i in range(0, 30):
    print(pos_tagged_corpus[i])

In [ ]:
# 동사 원형 복원 기준
#NNG(일반 명사)|NNP(고유 명사)|NNB(의존 명사) + XSV(동사 파생 접미사)|XSA(형용사 파생 접미사) --> NNG(일반 명사)|NNP(고유 명사)|NNB(의존 명사) + XSV(동사 파생 접미사)|XSA(형용사 파생 접미사) + 다
#NNG(일반 명사)|NNP(고유 명사)|NNB(의존 명사) + XSA(형용사 파생 접미사) + VX(보조 용언) --> NNG(일반 명사)|NNP(고유 명사) + XSA(형용사 파생 접미사) + 다
#VV(동사) --> VV(동사) + 다
#VX(보조 용언) --> VX(보조 용언) + 다
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.') # 명사 + 접미사
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX') # 명사 + 형용사 파생 접미사 + 보조 용언
p3 = re.compile('[가-힣A-Za-z0-9]+/VV') # 동사
p4 = re.compile('[가-힣A-Za-z0-9]+/VX') # 보조 용언

In [ ]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent) # 위에 컴파일한 것과 일치하는 경우 찾기
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '): # 공백 기준 분리
                lemma = term.split('/')[0] # 단어
                tag = term.split('/')[-1] # 품사
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

In [ ]:
# 함수 실행
stemming_corpus = stemming_text(pos_tagged_corpus)

for i in range(0, 30):
    print(stemming_corpus[i])

In [ ]:
# 불용어 설정, 한국어 코퍼스 최빈도어
stopwords = ['이/VCP','나오/VV','있/VA','가지/VV','하/VV','씨/NNB','것/NNB','시키/XSV','들/XSN',
'만들/VV','그/MM','지금/NNG'	,'되/VV','생각하/VV','수/NNB','그러/VV'	,'이/NP','속/NNG','보/VX',
'하나/NR','않/VX','집/NNG','없/VA','살/VV','나/NP','모르/VV','사람/NNG','적/XSN','주/VV','월/NNB',
'아니/VCN','데/NNB','등/NNB','자신/NNG','같/VA','안/MAG','우리/NP','어떤/MM','때/NNG','내/NP','년/NNB',
'내/VV','가/VV','경우/NNG','한/MM','명/NNB','지/VX','생각/NNG','대하/VV','시간/NNG','오/VV','그녀/NP',
'말/NNG','다시/MAG','일/NNG','이런/MM','그렇/VA','앞/NNG','위하/VV','보이/VV','때문/NNB','번/NNB',
'그것/NP','나/VX','두/VV','다른/MM','말하/VV','어떻/VA','알/VV','여자/NNG','그러나/MAJ','개/NNB',
'받/VV','전/NNG','못하/VX','들/VV','일/NNB','사실/NNG','그런/MM','이렇/VA','또/MAG','점/NNG','문제/NNG',
'싶/VX','더/MAG','말/VX','사회/NNG','정도/NNG','많/VA','좀/MAG','그리고/MAJ','원/NNB','좋/VA','잘/MAG',
'크/VA','통하/VV','따르/VV','소리/NNG','중/NNB','놓/VX']

In [ ]:
def remove_stopword_text(text):
    corpus = []
    for sent in text:
        modi_sent = []
        for word in sent.split(' '): # 공백 기준 분리
            if word not in stopwords:
                modi_sent.append(word) # 불용어가 아닌 단어들을 modi_sent에 추가
        corpus.append(' '.join(modi_sent)) # list에 공백을 구분자로 문자열로 변환
    return corpus

In [ ]:
# 함수 실행
removed_stopword_corpus = remove_stopword_text(stemming_corpus)

for i in range(0, 30):
    print(removed_stopword_corpus[i])